In [176]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT10] Series de Tiempo\Clases prácticas\Práctica 1-20250419\var.xlsx"
df = pd.read_excel(path)
df.head()

,dateid01,dateid,fed_funds,inflation,un_rate
0,1947-01-01,1947-03-31 23:59:59.999,NaN,NaN,NaN
1,1947-04-01,1947-06-30 23:59:59.999,NaN,5.737548,NaN
2,1947-07-01,1947-09-30 23:59:59.999,NaN,7.118936,NaN
3,1947-10-01,1947-12-31 23:59:59.999,NaN,8.188125,NaN
4,1948-01-01,1948-03-31 23:59:59.999,NaN,4.503751,3.733333


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   dateid01   219 non-null    datetime64[ns]
 1   dateid     219 non-null    datetime64[ns]
 2   fed_funds  189 non-null    float64       
 3   inflation  218 non-null    float64       
 4   un_rate    215 non-null    float64       
dtypes: datetime64[ns](2), float64(3)
memory usage: 8.7 KB


In [178]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   dateid01   189 non-null    datetime64[ns]
 1   dateid     189 non-null    datetime64[ns]
 2   fed_funds  189 non-null    float64       
 3   inflation  189 non-null    float64       
 4   un_rate    189 non-null    float64       
dtypes: datetime64[ns](2), float64(3)
memory usage: 7.5 KB


In [179]:
df['dateid'] = pd.to_datetime(df['dateid'])
df['dateidQ'] = df['dateid'].dt.to_period('Q')
df.sort_values(by=['dateid'], inplace=True)
df

,dateid01,dateid,fed_funds,inflation,un_rate,dateidQ
0,1954-07-01,1954-09-30 23:59:59.999,1.026667,0.000000,5.966667,1954Q3
1,1954-10-01,1954-12-31 23:59:59.999,0.986667,0.411100,5.333333,1954Q4
2,1955-01-01,1955-03-31 23:59:59.999,1.343333,1.640187,4.733333,1955Q1
3,1955-04-01,1955-06-30 23:59:59.999,1.500000,2.244331,4.400000,1955Q2
4,1955-07-01,1955-09-30 23:59:59.999,1.940000,2.838329,4.100000,1955Q3
...,...,...,...,...,...,...
184,2000-07-01,2000-09-30 23:59:59.999,6.520000,1.868115,4.066667,2000Q3
185,2000-10-01,2000-12-31 23:59:59.999,6.473333,1.748108,3.966667,2000Q4
186,2001-01-01,2001-03-31 23:59:59.999,5.593333,3.215838,4.233333,2001Q1
187,2001-04-01,2001-06-30 23:59:59.999,4.326667,2.092996,4.500000,2001Q2


In [180]:
df_f = df[(df['dateidQ']>='1960Q1')&(df['dateidQ']<='2000Q4')]
df_f.reset_index(drop=True, inplace=True)
df_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   dateid01   164 non-null    datetime64[ns]
 1   dateid     164 non-null    datetime64[ns]
 2   fed_funds  164 non-null    float64       
 3   inflation  164 non-null    float64       
 4   un_rate    164 non-null    float64       
 5   dateidQ    164 non-null    period[Q-DEC] 
dtypes: datetime64[ns](2), float64(3), period[Q-DEC](1)
memory usage: 7.8 KB


In [181]:
df_f = df_f[['dateid', 'fed_funds', 'inflation', 'un_rate']].copy()
df_f.set_index('dateid', inplace=True)

from statsmodels.tsa.api import VAR

# Crear el modelo VAR.
model = VAR(df_f, freq='QE')
results = model.fit(maxlags=4)
print(results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 19, Apr, 2025
Time:                     20:06:22
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -2.50944
Nobs:                     160.000    HQIC:                  -2.95464
Log likelihood:          -381.369    FPE:                  0.0384676
AIC:                     -3.25902    Det(Omega_mle):       0.0304310
--------------------------------------------------------------------
Results for equation fed_funds
                  coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------
const                0.540692         0.333943            1.619           0.105
L1.fed_funds         0.946091         0.091146           10.380           0.000
L1.inflation         0.068137         0.073169            0.